In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from scipy.sparse import csr_matrix

In [2]:

import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from scipy.sparse import csr_matrix

def item_based_knn_train(df_train, k=5):
    df_train["relevance"] = 5 * df_train["booking_bool"] + df_train["click_bool"]
    item_encoder = LabelEncoder()
    encoded_items = item_encoder.fit_transform(df_train["prop_id"])
    encoded_users = LabelEncoder().fit_transform(df_train["srch_id"])

    interaction_matrix = csr_matrix(
        (df_train["relevance"], (encoded_items, encoded_users)),
        shape=(len(np.unique(encoded_items)), len(np.unique(encoded_users)))
    )

    model = NearestNeighbors(n_neighbors=k+1, metric="cosine", algorithm="brute")
    model.fit(interaction_matrix)

    item_id_to_index = {pid: item_encoder.transform([pid])[0] for pid in df_train["prop_id"].unique()}
    index_to_item_id = {v: k for k, v in item_id_to_index.items()}

    return model, item_id_to_index, index_to_item_id

def score_test_using_item_knn(df_test, model, item_id_to_index, index_to_item_id, df_train):
    df_test["relevance"] = 5 * df_test.get("booking_bool", 0) + df_test.get("click_bool", 0)
    df_test["item_knn_score"] = 0.0
    popularity = df_train.groupby("prop_id")["relevance"].mean().to_dict()

    for i, row in df_test.iterrows():
        prop_id = row["prop_id"]
        if prop_id in item_id_to_index:
            idx = item_id_to_index[prop_id]
            distances, indices = model.kneighbors([model._fit_X[idx].toarray().flatten()])
            similar_props = [index_to_item_id[ind] for ind in indices.flatten() if ind != idx]
            score = df_train[df_train["prop_id"].isin(similar_props)]["relevance"].mean()
            if pd.isna(score):
                score = popularity.get(prop_id, 0)
        else:
            score = popularity.get(prop_id, 0)
        df_test.at[i, "item_knn_score"] = score

    df_test["item_knn_score"] = MinMaxScaler().fit_transform(df_test[["item_knn_score"]])
    return df_test

def dcg(relevances, k):
    relevances = np.array(relevances)[:k]
    return np.sum((2 ** relevances - 1) / np.log2(np.arange(2, len(relevances) + 2)))

def ndcg_at_k(relevances, k):
    ideal = sorted(relevances, reverse=True)
    ideal_dcg = dcg(ideal, k)
    return dcg(relevances, k) / ideal_dcg if ideal_dcg > 0 else 0.0

def mean_ndcg_at_k(df, score_column="item_knn_score", k=5):
    df = df.sort_values(["srch_id", score_column], ascending=[True, False])
    ndcgs = []
    for _, group in df.groupby("srch_id"):
        relevances = group["relevance"].tolist()
        ndcgs.append(ndcg_at_k(relevances, k))
    return np.mean(ndcgs)



In [3]:

df = pd.read_parquet("data/training_set_processed.parquet")
df_sample = df.sample(frac=0.1, random_state=42)

split = int(0.8 * len(df_sample))
df_train = df_sample[:split].copy()
df_val = df_sample[split:].copy()

model, item_id_to_index, index_to_item_id = item_based_knn_train(df_train, k=5)
df_val = score_test_using_item_knn(df_val, model, item_id_to_index, index_to_item_id, df_train)

ndcg = mean_ndcg_at_k(df_val, score_column="item_knn_score", k=5)
print(f"Validation NDCG@5: {ndcg:.4f}")

df_val.sort_values(by=["srch_id", "item_knn_score"], ascending=[True, False], inplace=True)
df_val[["srch_id", "prop_id"]].to_csv("item_knn_prediction_table.csv", index=False)
df_val.head()


Validation NDCG@5: 0.0536


,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,...,comp5_rate_percent_diff_flag,comp6_rate_percent_diff_flag,comp7_rate_percent_diff_flag,comp8_rate_percent_diff_flag,prop_review_score_zero_flag,prop_log_historical_price_zero_flag,result_count,prop_in_visitor_country,relevance,item_knn_score
15,1,12,187,NaN,NaN,219,85728,2,3.0,1,...,0,1,1,1,0,0,28,0,0,0.248062
26,1,12,187,NaN,NaN,219,114766,2,3.5,1,...,1,1,1,1,0,0,28,0,0,0.248062
442,43,5,219,NaN,NaN,219,31166,2,3.0,1,...,1,1,1,1,0,0,32,1,0,0.248062
542,46,5,77,NaN,NaN,30,85220,4,4.5,0,...,0,1,1,1,0,0,32,0,1,0.262654
666,57,16,31,NaN,NaN,31,7330,0,5.0,0,...,1,1,1,1,0,0,28,1,0,0.000000
